In [160]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as npr

## Turnbull & Wakeman approximation

Turnbull & Wakeman have provided an approximation of the price of an asian option, as a european option with different parameters.

In [161]:
def cdf(x):
    """ Approximation of the normal distribution function with an error less than 7.5*10^-8 """
    coefficients = [0.2316419, 0.319381530, -0.356563782, 1.781477937, -1.821255978, 1.33027442]
    if x >= 0:
        t = 1 / (1 + coefficients[0] * x)
        terms = [coefficients[i] * t**i for i in range(1, 6)]
        approx = 1 - (1 / np.sqrt(2 * np.pi)) * np.exp(-x**2 / 2) * sum(terms)
        return approx
    else:
        return 1 - cdf(-x)

In [162]:
def tnw_coeffs_con(r, T, sig):
    """ Calculates the r_a and sigma_a coefficients from the Turnbull & Wakeman approximation for an asian option """
    M1 = (np.exp(r*T)-1)/(r*T)
    M2 = ( 
        (2*np.exp((2*r+sig**2)*T)) / ((r+sig**2) * (2*r+sig**2) * T**2)
        + (2/(r*T**2)) * (1/(2*r+sig**2) - np.exp(r*T) / (r+sig**2))
    )

    r_a = np.log(M1) / T 
    sig_a = np.sqrt(-2*r_a + np.log(M2)/T)
    return (r_a, sig_a)

In [163]:
def tnw_coeffs_dis(r, T, sig, N):
    """ Calculates the r_a and sigma_a coefficients from the Turnbull & Wakeman approximation for an asian option """
    dt = T/N
    t0 = np.exp((2*r+sig**2)*dt)
    t1 = np.exp(r*dt)
    M1 = (1 / N)*( (t1*(1-np.exp(r*T))) / (1-t1) )
    M2 = (
        (1/(N**2))*( (t0 * (1-np.exp((2*r+sig**2)*T)) ) / (1-t0) )
        + (1/(N**2))*(2*t1/(1-t1))*( 
        t0 * ( (1-np.exp((2*r+sig**2)*(N-1)*dt)) / (1-t0) )
        - np.exp(((N+1)*r+sig**2)*dt) * ( (1-np.exp((r+sig**2)*(N-1)*dt)) / (1-np.exp((r+sig**2)*dt)) )
    ))

    r_A = np.log(M1) / T
    sig_a = np.sqrt(np.log(M2)/T - 2 * r_A)

    return (r_A, sig_a)

In [164]:
def asian_tnw_bs(r, T, S0, sig, K, N):
    """ Calculates price of an asian option under the Turnbull & Wakeman approximation and the Black-Scholes model """
    r_a, sig_a = tnw_coeffs_dis(r, T, sig, N)
    
    d = (np.log(S0 / K) + r_a * T + 0.5 * sig_a**2 * T) / (sig_a * np.sqrt(T))
    price = np.exp(-r * T) * (
        S0 * np.exp(r_a * T) * cdf(sig_a * np.sqrt(T) - d)
        - K * cdf(-d)
    )
    return price

In [165]:
def asian_mc_bs(r, T, S0, sig, K, N, n=1000):
    step = (T/N)
    LR = (r-sig**2/2) * step + sig*np.sqrt(step)*npr.normal(0, 1, size=(n, N))
    LR = np.concatenate((np.log(S0)*np.ones((n, 1)), LR), axis=1)
    LogPath = np.cumsum(LR, axis=1)
    SPath = np.exp(LogPath)

    payoffs_asian = np.exp(-r*T)*np.maximum(np.sum(SPath, axis=1)/N - K, 0)
    expected = payoffs_asian.mean()
    var = np.linalg.norm(payoffs_asian - expected) / n

    return payoffs_asian.mean(), var

In [166]:
K = 1.0
S0 = 1.0
sig = 0.3
T = 6
r = 0.01
N = 1512

tnw_approx = asian_tnw_bs(r, T, S0, sig, K, N)
mc_approx, var = asian_mc_bs(r, T, S0, sig, K, N)

In [167]:
tnw_approx, mc_approx, var

(0.17877088673623084, 0.19642700056934673, 0.011904141381981448)

In [168]:
mat = npr.random(size=(5,3))
mat

array([[0.12552939, 0.87967905, 0.47102611],
       [0.3014681 , 0.29998178, 0.64514324],
       [0.55184287, 0.82239473, 0.26819472],
       [0.36980535, 0.04764278, 0.85006542],
       [0.44849495, 0.33822962, 0.7277291 ]])

In [169]:
np.sum(mat, axis=1)

array([1.47623455, 1.24659312, 1.64243232, 1.26751355, 1.51445367])